<a href="https://githubtocolab.com/gee-community/geemap/blob/master/examples/notebooks/12_zonal_statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [1]:
# !pip install geemap

Load libraries

In [5]:
import ee
import geemap
import os
ee.Initialize()

Add layers

In [6]:
##added new layers
poly = ee.FeatureCollection('projects/ee-tmasaki040685/assets/eth-admin1')
esri = ee.ImageCollection(
    "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS"
).filterDate('2017-01-01','2017-12-31').mosaic().clip(poly)
##.filterDate() specifies date
##.mosaic() turn it into image
##.clip() masks image to the boundary of poly


esri_vis = {
    'min': 1,
    'max': 11,
    'palette': [
        "#1A5BAB",
        "#358221",
        "#000000",
        "#87D19E",
        "#FFDB5C",
        "#000000",
        "#ED022A",
        "#EDE9E4",
        "#F2FAFF",
        "#C8C8C8",
        "#C6AD8D"
    ],
}

legend_dict = {
    '1 Water': '#1A5BAB',
    '2 Trees': '#358221',
    '4 Flooded Vegetation': '#87D19E5',
    '5 Crops': '#FFDB5C',
    '7 Built area': '#ED022A',
    '8 Bare ground': '#EDE9E4',
    '9 Snow/ice': '#F2FAFF',
    '10 Clouds': '#C8C8C8',
    '11 Rangeland': '#C6AD8D',
}

Generate map

In [3]:
Map = geemap.Map()
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [4]:
region = ee.Geometry.BBox(32.999840, 3.402422, 32.999840, 14.963733)
Map.addLayer(esri, esri_vis, "ESRI Land Cover")
Map.add_legend(legend_title="ESRI Land Cover Classification", legend_dict=legend_dict)
Map.centerObject(region)

EEException: Collection.loadTable: Collection asset 'projects/ee-tmasaki040685/assets/eth-admin1' not found.

Export image (but this does not run because the extent of region for which image is being analyzed is too large...). 

In [39]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'esri_lulc10.tif')

feature = Map.draw_last_feature

if feature is None:
    geom = ee.Geometry.Polygon(
        [
            [
                [32.999840, 3.402422],
                [32.999840, 14.963733],
                [47.990990, 3.402422],
                [47.990990, 14.963733],
                [32.999840, 3.402422],
            ]
        ]
    )
    feature = ee.Feature(geom, {})

roi = feature.geometry()

geemap.ee_export_image(
    esri, filename=filename, scale=90, region=roi, file_per_band=False
)

Generating URL ...
An error occurred while downloading.
Total request size (530395488 bytes) must be less than or equal to 50331648 bytes.


In [7]:
Map.addLayer(poly, {}, 'eth')

Generate a new layer coded 1 if cropland

In [40]:
esri_crop = esri.eq(5).selfMask().rename('binary')

In [41]:
region = ee.Geometry.BBox(32.999840, 3.402422, 32.999840, 14.963733)

esri_crop_vis = {
    'min': 0,
    'max': 1,
    'palette': [
        "#000000",
        "#FFDB5C"
    ],
}

Map.addLayer(esri_crop, esri_crop_vis, "ESRI Land Cover")
Map.add_legend(legend_title="ESRI Land Cover Classification", legend_dict=legend_dict)
Map.centerObject(region)

Compute zonal statistics

In [48]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_esri_lulc10_stats = os.path.join(out_dir, 'esri_lulc10_5.csv')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(esri_crop, poly, out_esri_lulc10_stats, statistics_type='COUNT', scale=100)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\WB495141\Downloads\esri_lulc10_5.csv


In [49]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_esri_lulc10_stats = os.path.join(out_dir, 'esri_lulc10_all.csv')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(esri, poly, out_esri_lulc10_stats, statistics_type='COUNT', scale=100)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\WB495141\Downloads\esri_lulc10_all.csv
